# Ion density boundary for Mach cones

This notebook analyses the distribution of plasma ions around the Mach cone boundary. Scientific case of NorSat-1 satellite in a supersonic plasma flow.

Group 3: ***Aurora***
 - Doyeon Kim
 - Michał Jan Odorczuk
 - Rachel Knutsen Stiansen
 - Shunsuke Kawamura
 - Taito Taniguchi

### Description

The simulation parameters were set to:

| Parameter | default|
| :--- | ---:|
| Plasma density | $10^5$ #/cc|
| Electron temperature | $1000$ K|
| Ion temperature | $750$ K|
| Ion to electron mass ratio | $1000$|

with plasma flow in the positive x direction. All plots include the information on the used the Mach number and the magnetic field orientation and strength. All simulation used [EMSES](https://www.energy.kyoto-u.ac.jp/gcoe/en/emses/aboutemses.htm) software on CAMPHOR supercomputer from [Kyoto University](https://www.kyoto-u.ac.jp/ja).

The simulations used a mesh of $160 \times 160 \times 154$, with the grid width of $0.5$ cm. The mesh included a simplified model of the NorSat-1 satellite consisting of 9 elements:
 - Main body - cuboid stretching from $(59,74,63)$ to $(80,86,93)$,
 - Solar panel 1 - cuboid stretching from $(59, 62, 63)$ to $(60, 74, 93)$,
 - Solar panel 2 - cuboid stretching from $(59, 86, 63)$ to $(60, 98, 93)$,
 - Langmuir probe rod 1 - cylinder of radius $0.5$, aligned with the y-axis, stretching from $(80, 51, 63)$ to $(80, 109, 63)$,
 - Langmuir probe rod 2 - cylinder of radius $0.5$, aligned with the y-axis, stretching from $(80, 51, 93)$ to $(80, 109, 93)$,
 - Langmuir probe ball 1 - sphere of radius $0.5$, centered at $(80, 110, 63)$,
 - Langmuir probe ball 2 - sphere of radius $0.5$, centered at $(80, 110, 93)$,
 - Langmuir probe ball 3 - sphere of radius $0.5$, centered at $(80, 50, 63)$,
 - Langmuir probe ball 4 - sphere of radius $0.5$, centered at $(80, 50, 93)$,

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from os import listdir
import re

## Data

All the datafiles should be located in `./res/data/` and their names should be in the format `data_B<x>_Ma<y>_<z>uT.csv`, where `<x>` is the direction of the magnetic field, `<y>` is the Mach number of the flow, and `<z>` is the magnetic field strength in microteslas. The files are expected to be in the format as exported from Paraview, with at least six columns:

| Variable name | Type | Description |
| :--- | :--- | ---: |
| `Structured Coordinates:0` | integer | Point coordinate in the x-direction |
| `Structured Coordinates:1` | integer | Point coordinate in the y-direction |
| `Structured Coordinates:2` | integer | Point coordinate in the z-direction |
| `nd1p` | floating point number | Normalised number density of electrons |
| `nd2p` | floating point number | Normalised number density of ions |
| `phisp` | floating point number | Normalised electric field potential |

In [2]:
data_dir = "./res/data/"
dirs = listdir(data_dir)
dfs = {}
files = [file for file in dirs if file[:4] == "data"]
for file in files:
    label = re.search("^data_(.*)\.csv$", file).group(1)
    dfs[label] = pd.read_csv(data_dir + file)
    print("simulation with parameters ", label, " included")

simulation with parameters  Bz_Ma6.4_50uT  included
simulation with parameters  Bx_Ma0.2_0uT  included
simulation with parameters  Bx_Ma2.0_70uT  included
simulation with parameters  Bx_Ma4.0_50uT  included
simulation with parameters  Bx_Ma2.0_100uT  included
simulation with parameters  Bz_Ma0.2_50uT  included
simulation with parameters  Bx_Ma2.0_50uT  included
simulation with parameters  Bx_Ma2.0_0uT  included
simulation with parameters  Bx_Ma0.2_50uT  included
simulation with parameters  Bx_Ma6.4_50uT  included
simulation with parameters  Bx_Ma4.0_0uT  included
simulation with parameters  Bz_Ma2.0_50uT  included
simulation with parameters  Bx_Ma6.4_0uT  included


## Function definitions

### Background for plots

Displaying field data as a background for the plot.

 - `df` - dataframe containing the background data. It has to include three coordinates `Structured Coordinates:0`, `Structured Coordinates:1` and `Structured Coordinates:2`, and the background field data under label given by attribute `field`.
 - `ax` - axis object for plotting,
 - `xlow` - lower limit of the plot in x-direction (`Structured Coordinates:0`),
 - `xhigh` - upper limit of the plot in x-direction (`Structured Coordinates:0`),
 - `ylow` - lower limit of the plot in y-direction (`Structured Coordinates:1`),
 - `yhigh` - upper limit of the plot in y-direction (`Structured Coordinates:1`),
 - `z` - z (`Structured Coordinates:1`) coordinate of the slicing plane,
 - `field` - field to be displayed as the background,
 - `steps` - number of contour levels to be displayed over the plot.

In [3]:
def background(df, ax, xlow, xhigh, ylow, yhigh, z, field, steps=10):
    xs = df["Structured Coordinates:0"]
    ys = df["Structured Coordinates:1"]
    zs = df["Structured Coordinates:2"]
    df = df[(zs==z)&(xs>=xlow)&(xs<=xhigh)&(ys>=ylow)&(ys<=yhigh)]
    phi = df[field]
    grid_x = np.unique(df["Structured Coordinates:0"])
    grid_y = np.unique(df["Structured Coordinates:1"])

    X, Y = np.meshgrid(grid_x, grid_y)
    PHI = phi.to_numpy().reshape(X.shape)

    ax.pcolor(X, Y, PHI, cmap=plt.get_cmap("gray"))
    min_phi, max_phi = PHI.min(), PHI.max()
    ax.contour(X, Y, PHI, np.arange(min_phi, max_phi, (max_phi - min_phi) / steps), cmap=plt.get_cmap("gray"))

### Outer cone

Object moving through a medium supersonic will produce a shock wave downstream. The upstream-downstream boundary should be sharp and easily distinguishible. By marking sudden drops in the ion density, we can estimate the shape of the cone. Because of a subsonic wake in front of the spacecraft, we should assume the cone to stabilise only slightly behind the spacecraft's front. For that, let us...